# Step 1.1: Understanding our dataset

In [1]:
import pandas as pd
# Dataset from - https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
df = pd.read_csv( 'SMSSpamCollection',sep='\t',header= None, names=['label','sms_message'])

# Output printing out first 5 columns
df.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Step 1.2: Data Preprocessing

In [2]:
 #lets convert our labels to binary variables, 0 to represent 'ham'(i.e. not spam) and 1 to represent 'spam' for ease of computation.

#as Scikit-learn only deals with numerical values 

df['label'] = df['label'].map({'ham':0, 'spam':1})
print(df.shape)
df.append(df['label'])
df.head()

(5572, 2)


,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


# Step 2.1: Bag of words

In [5]:
#The basic idea of BoW is to take a piece of text and count the frequency of the words in that text. 
#It is important to note that the BoW concept treats each word individually and the order in which the words occur does not matter.
documents = ['Hello, how are you!',
                'Win money, win from home.',
                'Call me now.',
                'Hello, Call hello you tomorrow?']

from sklearn.feature_extraction.text import CountVectorizer
count_vector= CountVectorizer()
count_vector.fit(documents)
count_vector.get_feature_names()


['are',
 'call',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'tomorrow',
 'win',
 'you']

In [7]:
#Create a matrix with the rows being each of the 4 documents, and the columns being each word.
 #The corresponding (row, column) value is the frequency of occurrence of that word(in the column) in a particular document(in the row). 
doc_array =count_vector.transform(documents).toarray()# TODO
doc_array

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1]])

In [8]:
#To make it easier to understand our next step is to convert this array into a dataframe and name the columns appropriately.
frequency_matrix = pd.DataFrame(data=doc_array,index=documents,columns=['are',
 'call',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'tomorrow',
 'win',
 'you'])
frequency_matrix

,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
"Hello, how are you!",1,0,0,1,0,1,0,0,0,0,0,1
"Win money, win from home.",0,0,1,0,1,0,0,1,0,0,2,0
Call me now.,0,1,0,0,0,0,1,0,1,0,0,0
"Hello, Call hello you tomorrow?",0,1,0,2,0,0,0,0,0,1,0,1


# Step 3.1: Training and testing sets

In [9]:
# splitting into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['sms_message'], 
                                                    df['label'], 
                                                    random_state=1)

print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 5572
Number of rows in the training set: 4179
Number of rows in the test set: 1393


# Step 3.2: Applying Bag of Words processing to our dataset

In [12]:
#Firstly, we have to fit our training data (X_train) into CountVectorizer() and return the matrix.
#Secondly, we have to transform our testing data (X_test) to return the matrix.
#X_train is our training data for the 'sms_message' column in our dataset and we will be using this to train our model.

#X_test is our testing data for the 'sms_message' column and this is the data we will be using(after transformation to a matrix) to make predictions on. 
# Instantiate the CountVectorizer method
count_vector = CountVectorizer()

# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

training_data


<4179x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 55209 stored elements in Compressed Sparse Row format>

# Step 4:Naive Bayes implementation using scikit-learn

In [13]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
#making predictions
predictions = naive_bayes.predict(testing_data)
predictions

array([0, 0, 0, ..., 0, 1, 0])

# Step 5: Evaluating our model

In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test,predictions)))
print('Precision score: ', format(precision_score(y_test,predictions)))
print('Recall score: ', format(recall_score(y_test,predictions)))
print('F1 score: ', format(f1_score(y_test,predictions)))

Accuracy score:  0.9885139985642498
Precision score:  0.9720670391061452
Recall score:  0.9405405405405406
F1 score:  0.9560439560439562
